In [1]:
import pandas as pd
from mlxtend.frequent_patterns import association_rules , apriori

# Read Dataset

In [2]:
sales_reciepts = pd.read_csv("dataset/201904 sales reciepts.csv")

In [20]:
sales_reciepts

,transaction_id,transaction_date,sales_outlet_id,customer_id,product_id,quantity
0,7,2019-04-01,3,558,52,1
1,11,2019-04-01,3,781,27,2
2,19,2019-04-01,3,788,46,2
3,32,2019-04-01,3,683,23,2
4,33,2019-04-01,3,99,34,1
...,...,...,...,...,...,...
49889,753,2019-04-29,8,0,30,1
49890,756,2019-04-29,8,8412,25,2
49891,759,2019-04-29,8,0,31,1
49892,763,2019-04-29,8,8030,44,2


In [5]:
product = pd.read_csv("dataset/product.csv")
product.head()

,product_id,product_group,product_category,product_type,product,product_description,unit_of_measure,current_wholesale_price,current_retail_price,tax_exempt_yn,promo_yn,new_product_yn
0,1,Whole Bean/Teas,Coffee beans,Organic Beans,Brazilian - Organic,It's like Carnival in a cup. Clean and smooth.,12 oz,14.40,$18.00,Y,N,N
1,2,Whole Bean/Teas,Coffee beans,House blend Beans,Our Old Time Diner Blend,Out packed blend of beans that is reminiscent ...,12 oz,14.40,$18.00,Y,N,N
2,3,Whole Bean/Teas,Coffee beans,Espresso Beans,Espresso Roast,Our house blend for a good espresso shot.,1 lb,11.80,$14.75,Y,N,N
3,4,Whole Bean/Teas,Coffee beans,Espresso Beans,Primo Espresso Roast,Our primium single source of hand roasted beans.,1 lb,16.36,$20.45,Y,N,N
4,5,Whole Bean/Teas,Coffee beans,Gourmet Beans,Columbian Medium Roast,A smooth cup of coffee any time of day.,1 lb,12.00,$15.00,Y,N,N


# Data Wrangling

## Merge Data

In [7]:
sales_reciepts = sales_reciepts[["transaction_id" , "transaction_date" , "sales_outlet_id" , "customer_id" ,"product_id" ,"quantity"]]
product = product[["product_id" , "product_category" , "product"]]

dataset = pd.merge(sales_reciepts , product , on = "product_id" ,how="left")
dataset.head()

,transaction_id,transaction_date,sales_outlet_id,customer_id,product_id,quantity,product_category,product
0,7,2019-04-01,3,558,52,1,Tea,Traditional Blend Chai Rg
1,11,2019-04-01,3,781,27,2,Coffee,Brazilian Lg
2,19,2019-04-01,3,788,46,2,Tea,Serenity Green Tea Rg
3,32,2019-04-01,3,683,23,2,Coffee,Our Old Time Diner Blend Rg
4,33,2019-04-01,3,99,34,1,Coffee,Jamaican Coffee River Sm


## Remove sizes

In [8]:
dataset[dataset["product"].str.contains("Dark chocolate")]["product"].unique()

array(['Dark chocolate Lg', 'Dark chocolate Rg', 'Dark chocolate'],
      dtype=object)

In [9]:
dataset["product"].nunique()

80

In [10]:
dataset["product"] = dataset["product"].str.replace(" Rg" , "")
dataset["product"] = dataset["product"].str.replace(" Sm" , "")
dataset["product"] = dataset["product"].str.replace(" Lg" , "")

In [11]:
dataset["product"].nunique()

45

In [12]:
print(dataset["product"].nunique())

45


### Choose Product Subset

In [13]:
products_to_take = ['Cappuccino', 'Latte', 'Espresso shot',  \
                     'Dark chocolate','Sugar Free Vanilla syrup', 'Chocolate syrup',\
                    'Carmel syrup', 'Hazelnut syrup', 'Ginger Scone',  \
                    'Chocolate Croissant', 'Jumbo Savory Scone', 'Cranberry Scone', 'Hazelnut Biscotti',\
                    'Croissant', 'Almond Croissant', 'Oatmeal Scone', 'Chocolate Chip Biscotti',\
                    'Ginger Biscotti',\
                   ]


In [14]:
dataset = dataset[dataset["product"].isin(products_to_take)]

In [15]:
dataset.head()

,transaction_id,transaction_date,sales_outlet_id,customer_id,product_id,quantity,product_category,product
16,108,2019-04-01,3,65,40,1,Coffee,Cappuccino
17,112,2019-04-01,3,90,37,2,Coffee,Espresso shot
20,127,2019-04-01,3,116,41,2,Coffee,Cappuccino
21,134,2019-04-01,3,189,38,2,Coffee,Latte
22,135,2019-04-01,3,131,40,1,Coffee,Cappuccino


In [16]:
dataset[["product" , "product_category"]].drop_duplicates().reset_index(drop=True)

,product,product_category
0,Cappuccino,Coffee
1,Espresso shot,Coffee
2,Latte,Coffee
3,Dark chocolate,Drinking Chocolate
4,Oatmeal Scone,Bakery
5,Jumbo Savory Scone,Bakery
6,Chocolate Chip Biscotti,Bakery
7,Ginger Biscotti,Bakery
8,Chocolate Croissant,Bakery
9,Hazelnut Biscotti,Bakery


### Clean Transactions

In [17]:
dataset["transaction"] = dataset["transaction_id"].astype(str) + "_" + dataset["customer_id"].astype(str)

In [18]:
dataset.head()

,transaction_id,transaction_date,sales_outlet_id,customer_id,product_id,quantity,product_category,product,transaction
16,108,2019-04-01,3,65,40,1,Coffee,Cappuccino,108_65
17,112,2019-04-01,3,90,37,2,Coffee,Espresso shot,112_90
20,127,2019-04-01,3,116,41,2,Coffee,Cappuccino,127_116
21,134,2019-04-01,3,189,38,2,Coffee,Latte,134_189
22,135,2019-04-01,3,131,40,1,Coffee,Cappuccino,135_131


In [19]:
num_of_items_for_each_transaction = dataset["transaction"].value_counts().reset_index()
num_of_items_for_each_transaction.head()

,transaction,count
0,209_0,31
1,206_0,30
2,204_0,27
3,208_0,25
4,203_0,24


In [21]:
num_of_items_for_each_transaction[num_of_items_for_each_transaction["count"]==1]

,transaction,count
2646,471_8410,1
2647,1011_0,1
2648,341_5545,1
2649,1627_8289,1
2650,328_5635,1
...,...,...
8381,135_523,1
8382,130_157,1
8383,121_465,1
8384,118_748,1


In [22]:
valid_transactions = num_of_items_for_each_transaction[(num_of_items_for_each_transaction["count"] > 1)]['transaction'].tolist()
valid_transactions[:10]

['209_0',
 '206_0',
 '204_0',
 '208_0',
 '203_0',
 '207_0',
 '205_0',
 '151_0',
 '323_0',
 '301_0']

In [23]:
dataset = dataset[dataset["transaction"].isin(valid_transactions)]

In [24]:
dataset.shape

(10189, 9)

### Product Trends

In [25]:
dataset["product_category"].value_counts()

product_category
Bakery                3800
Coffee                3174
Flavours              2246
Drinking Chocolate     947
Packaged Chocolate      22
Name: count, dtype: int64

In [26]:
dataset["product"].value_counts()

product
Cappuccino                  1290
Latte                       1256
Dark chocolate               969
Chocolate Croissant          636
Espresso shot                628
Sugar Free Vanilla syrup     605
Chocolate syrup              568
Carmel syrup                 561
Hazelnut syrup               512
Ginger Scone                 417
Jumbo Savory Scone           357
Croissant                    355
Chocolate Chip Biscotti      352
Cranberry Scone              350
Almond Croissant             347
Hazelnut Biscotti            338
Oatmeal Scone                334
Ginger Biscotti              314
Name: count, dtype: int64

## Popularity Recommendation engine

In [28]:
product_recommendation = dataset.groupby(["product" , "product_category"]).count().reset_index()

In [30]:
product_recommendation = product_recommendation[["product" , "product_category" , "transaction_id"]]
product_recommendation = product_recommendation.rename(columns={"transaction_id":"number_of_transactions"})
product_recommendation.head()

,product,product_category,number_of_transactions
0,Almond Croissant,Bakery,347
1,Cappuccino,Coffee,1290
2,Carmel syrup,Flavours,561
3,Chocolate Chip Biscotti,Bakery,352
4,Chocolate Croissant,Bakery,636


In [31]:
product_recommendation.to_csv("api/recommendation_objects/popularity_recommendation.csv" , index=False)

## Apriori Recommendation Engine